In [1]:
import numpy as np
import pickle
from matplotlib.patches import Ellipse
from sklearn import mixture
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.misc import imread
from scipy.misc import imshow
from matplotlib import animation
import datetime
from map_overlay import MapOverlay
import warnings
warnings.filterwarnings('ignore')

In [18]:
curr_dir = os.getcwd()
data_path = curr_dir + '/../data/'
fig_path = curr_dir + '/../figs/'
animation_path = curr_dir + '/../animation/'

In [3]:
with open(os.path.join(data_path, 'hourlyUtilization100_uncapped.pck'), 'rb') as f:
    load_data = pickle.load(f)
        
with open(os.path.join(data_path, 'ElementKeytoLatLong.pck'), 'rb') as f:
    locations = pickle.load(f)

In [4]:
# Average loads at each hour of the day for each day of the week for each block.
avg_loads = []

# Midpoint of lat and longs for each blockface.
gps_loc = []

# Block data at each hour for each blockface.
park_data = {}

elkeys = sorted(load_data.keys())

# Number of locations.
N = len(elkeys)

for key in elkeys:
    
    # Loading the data for a single block.
    block_data = pd.DataFrame(load_data[key].items(), columns=['Datetime', 'Load'])
    
    block_data['Datetime'] = pd.to_datetime(block_data['Datetime'])
    
    block_data.sort(columns='Datetime', inplace=True)
    block_data.reset_index(inplace=True, drop=True)
    
    block_data['Date'] = block_data['Datetime'].dt.date
    
    block_data['Hour'] = block_data['Datetime'].dt.hour
    
    block_data['Day'] = block_data['Datetime'].dt.weekday
    
    # Getting rid of sunday since there is no paid parking.
    block_data = block_data.loc[block_data['Day'] != 6]
    
    # Getting the dayes where the total parking is 0 because of holidays.
    empty_day_dates = []
    
    # No parking on new years eve.
    empty_day_dates.append(datetime.date(2015,1,1))
    
    # No parking on MLK day.
    empty_day_dates.append(datetime.date(2015,1,19))
    
    # No parking on Presidents day.
    empty_day_dates.append(datetime.date(2015,2,16))
    
    # Dropping the days where the total parking is 0.
    block_data = block_data.loc[~block_data['Date'].isin(empty_day_dates)]
    block_data.reset_index(inplace=True, drop=True)
    
    park_data[key] = block_data
    
    # Getting the average load for each hour of the week for the block.
    avg_load = block_data.groupby(['Day', 'Hour'])['Load'].mean().values.reshape((1,-1))
    avg_loads.append(avg_load)
    
    # Finding the midpoint of the GPS location of the block.
    mid_lat = (locations[key][0][0] + locations[key][1][0])/2.
    mid_long = (locations[key][0][1] + locations[key][1][1])/2.
    gps_loc.append([mid_lat, mid_long])
    
avg_loads = np.vstack((avg_loads))
gps_loc = np.vstack((gps_loc))

# Number of times.
P = avg_loads.shape[1]

In [13]:
model_selection = {"likelihood": [], "bic": [], "aic": []} 
min_comps = 1
max_comps = 20

for time in range(P):    
    likelihoods = []
    bics = []
    aics = []
    
    for num_comps in range(min_comps, max_comps):
        cluster_data = np.hstack((avg_loads[:, time, None], gps_loc))

        scaler = MinMaxScaler().fit(cluster_data)
        cluster_data = scaler.transform(cluster_data)
        
        gmm = mixture.GaussianMixture(n_init=10, n_components=num_comps, 
                                      covariance_type='diag').fit(cluster_data)
        
        likelihoods.append(gmm.lower_bound_)
        bics.append(gmm.bic(cluster_data))
        aics.append(gmm.aic(cluster_data))
        
    model_selection['likelihood'].append(likelihoods)
    model_selection['bic'].append(bics)
    model_selection['aic'].append(aics)

In [19]:
# Likelihood model selection plot.
plt.figure()
mean_likelihood = np.mean(np.vstack((model_selection['likelihood'])), axis=0)
plt.plot(range(min_comps, max_comps), mean_likelihood, 'o-', color='red')
plt.axvline(x=5, color='black')
plt.axvline(x=10, color='black')
plt.axvline(x=15, color='black')
plt.axvline(x=20, color='black')
plt.axvline(x=25, color='black')
plt.xlabel('Number of Components')
plt.ylabel('Likelihood')
plt.title('Likelihood Model Selection')
plt.savefig(os.path.join(fig_path, 'likelihood_model.png'))

# BIC model selection plot.
plt.figure()
mean_bic = np.mean(np.vstack((model_selection['bic'])), axis=0)
plt.plot(range(min_comps, max_comps), mean_bic, 'o-', color='red')
plt.axvline(x=5, color='black')
plt.axvline(x=10, color='black')
plt.axvline(x=15, color='black')
plt.axvline(x=20, color='black')
plt.axvline(x=25, color='black')
plt.xlabel('Number of Components')
plt.ylabel('BIC')
plt.title('BIC Model Selection')
plt.savefig(os.path.join(fig_path, 'bic_model.png'))

# AIC model selection plot.
plt.figure()
mean_aic = np.mean(np.vstack((model_selection['aic'])), axis=0)
plt.plot(range(min_comps, max_comps), mean_aic, 'o-', color='red')
plt.axvline(x=5, color='black')
plt.axvline(x=10, color='black')
plt.axvline(x=15, color='black')
plt.axvline(x=20, color='black')
plt.axvline(x=25, color='black')
plt.xlabel('Number of Components')
plt.ylabel('AIC')
plt.title('AIC Model Selection')
plt.savefig(os.path.join(fig_path, 'aic_model.png'))

In [20]:
def init_animation(gps_loc, N, fig_path):
    """Initializing figure for animation.
    
    :param gps_loc: numpy array, each row containing the lat and long of a block.
    :param N: The number of samples (blocks) in the data.
    :param fig_path: path to read background figure from.
    
    :return fig: figure object containing the loaded background image.
    :return ax: ax object.
    :return scatter: Matplotlib path collections object for plotting the block
    location centerpoints.
    :return scatter_centroid: Matplotlib path collections object for plotting 
    mixture centroids.
    :return mp: MapOverlay object for plotting over the map.
    :return center: Tuple of the center of the image with respect to gps coords.
    :return pix_center: List of the x and y pixel positions of the center of the image.
    """

    # Image specs to overlay plots on.
    upleft = [47.6197793,-122.3592749]
    bttmright = [47.607274, -122.334786]
    imgsize = [1135,864]

    # Setting up the image overlay class.
    mp = MapOverlay(upleft, bttmright, imgsize)

    # Converting the gps locations to pixel positions.
    pixpos = np.array([mp.to_image_pixel_position(list(gps_loc[i,:])) for i in range(N)])

    # Setting center of image.
    center = ((upleft[0] - bttmright[0])/2., (upleft[1] - bttmright[1])/2.)
    pix_center = mp.to_image_pixel_position(list(center))

    # Creating figure of map that will be plotted on top of.
    fig = plt.figure(figsize=(18,16))
    ax = plt.axes(xlim=(min(pixpos[:,0])-100, max(pixpos[:,0])+100), 
                  ylim=(min(pixpos[:,1])-100, max(pixpos[:,1])+100))

    ax.invert_yaxis()
    
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])

    im = imread(os.path.join(fig_path, "belltown.png"))
    ax.imshow(im)

    # Adding in the midpoints of the block faces to the map as points.
    scatter = ax.scatter(pixpos[:, 0], pixpos[:, 1], s=175, color='red')
    
    ax.xaxis.label.set_fontsize(25)
    ax.set_title('Gaussian Mixture Model on Average Load Distribution and Location', fontsize=25)
    
    scatter_centroid = ax.scatter([], [], s=500, color='red')

    return fig, ax, scatter, scatter_centroid, mp, center, pix_center

In [21]:
def animate(frame, times, ax, scatter, scatter_centroid, pathches, ellipses, 
            mp, default_means, center, pix_center, loads, gps_loc, num_comps):
    
    """Animating the mixture model for a set of times in a day.
    
    :param frame: The iteration number of the animation frame.
    :param times: List of time indexes to create the animation for.
    :param ax: ax object for plotting and labels.
    :param scatter: Matplotlib path collections object for plotting the block
    location centerpoints.
    :param scatter_centroid: Matplotlib path collections object for plotting 
    mixture centroids.
    :param patches: Matplotlib pathces objects for the mixture ellipses.
    :param ellipses: Ellipse patch objects for the mixture ellipses.
    :param mp: MapOverlay object for plotting over the map.
    :param default_means: Numpy array of the default locations for the 
    centroids, to attempt to keep the colors for the clusters the same.
    :param pix_center: List containing the x and y pixel centerpoints of the 
    image that is being overlayed on.
    :param center: Tuple of the center of the image with respect to gps coords.
    :param pix_center: List of the x and y pixel positions of the center of the image.
    :param loads: Numpy array containing the load data for the mixture model.
    :param gps_loc: Numpy array containing the gps locations for the mixture model.
    :param num_comps: Number of components to use for the mixture model.
    
    :return: Updated ax, scatter, scatter_centroid, scatter_centroid, pathches,
    and ellipses objects.
    """    
    
    # Getting the time to perform the mixture on for the frame iteration.
    time = times[frame]

    days = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday'}
    
    colors = [plt.cm.gist_rainbow(i) for i in np.linspace(0,1,num_comps)]
    
    cluster_data = np.hstack((loads[:, time, None], gps_loc))
    
    # Saving the cluster data prior to any scaling for plotting.
    cluster_data_true = cluster_data

    scaler = MinMaxScaler().fit(cluster_data)
    cluster_data = scaler.transform(cluster_data)
    
    gmm = mixture.GaussianMixture(n_init=2000, n_components=num_comps, 
                                  covariance_type='diag').fit(cluster_data)
    
    # Scaling the mean and covariances back to gps coordinates.
    means = np.vstack(([(mean[1:] - scaler.min_[1:])/(scaler.scale_[1:]) for mean in gmm.means_]))
    covs = np.dstack(([np.diag((cov[1:])/(scaler.scale_[1:]**2)) for cov in gmm.covariances_])).T
    
    labels = gmm.predict(cluster_data)    

    color_codes = {}
    for i in range(num_comps):
        
        # Finding the default centroid clostest to the current centroid.
        dists = [(j, np.linalg.norm(means[i] - default_means[j])) for j in range(num_comps)]
        best_colors = sorted(dists, key=lambda item:item[1])
        
        # Finding the color that is unused that is closest to the current centroid.
        unused_colors = [color[0] for color in best_colors if color[0] 
                         not in color_codes.values()]
        
        # Choosing the closest centroid that is not already used.
        choice = unused_colors[0]
        color_codes[i] = choice
    
    # Setting the cluster colors to keep the colors the same each iteration.
    scatter.set_color([colors[color_codes[labels[i]]] for i in range(len(labels))]) 
    
    num = 0
    
    # Updating the ellipses for each of the components.
    for i in range(num_comps):
        lambda_, v = np.linalg.eig(covs[i])
        lambda_ = np.sqrt(lambda_)
        
        # Getting the elipses for the 1st and 2nd standard deviations.
        for j in [1, 2]:
            
            # Converting mean in gps coords to pixel positions.
            xy = mp.to_image_pixel_position(list(means[i,:]))
            
            # Width and height of the elipses in gps coords.
            width = lambda_[0]*j*2
            height = lambda_[1]*j*2 
            
            # Center of the ellipse in pixel positions.
            new_center = (center[0]+width, center[1]+height)
            new_center = mp.to_image_pixel_position(list(new_center))
            
            # New width and height of the ellipses in pixel positions.
            width = abs(new_center[0] - pix_center[0])
            height = abs(new_center[1] - pix_center[1])
            
            # Updating the ellipses for the animation.
            patches[num].center = xy
            patches[num].width = width
            patches[num].height = height
            patches[num].edgecolor = colors[color_codes[i]]
            
            num += 1
            
    # Converting the centroids to pixel positions from gps coords.
    means = np.array([mp.to_image_pixel_position(list(means[i,:])) for i in range(len(means))])
    
    # Updating the centroids for the animations.
    scatter_centroid.set_offsets(means)
    
    hour = time % 10
    day = time/10
    
    ax.set_xlabel(days[day] + ' ' + str(8+hour) + ':00')
    
    return ax, scatter, scatter_centroid, scatter_centroid, pathches, ellipses

In [22]:
# The number of mixture components to use.
num_comps = 4

fig, ax, scatter, scatter_centroid, mp, center, pix_center = init_animation(gps_loc, N, fig_path)

# Creating the elipses for plotting purposes.
patches = [Ellipse(xy=(0, 0), width=0, height=0, angle=0, edgecolor='black', 
           facecolor='none', lw='4') for comp in range(2*num_comps)]

ellipses = [ax.add_patch(patches[comp]) for comp in range(2*num_comps)]

times = range(P)

default_means = np.array([[47.61348888, -122.34343007],[47.61179196, -122.34500616],
                          [47.61597088, -122.35054099],[47.61706817, -122.34617185]])

ani = animation.FuncAnimation(fig=fig, func=animate, frames=P, 
                              fargs=(times, ax, scatter, scatter_centroid, patches, 
                                     ellipses, mp, default_means, center, 
                                     pix_center, avg_loads, gps_loc, num_comps, ), 
                              interval=200)


FFwriter = animation.FFMpegWriter()
ani.save(os.path.join(animation_path, 'mixture.mp4'), writer = FFwriter)